In [34]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import zipfile
from sklearn.metrics import mean_squared_error

In [14]:
zip_path = '../00_Data_Bases/df_recommendersystem.zip' 
csv_filename = 'df_recommendersystem.csv'

with zipfile.ZipFile(zip_path, 'r') as z:
    with z.open(csv_filename) as f:
        df = pd.read_csv(f)

#### Matriz de interacción basada en clusters

In [15]:
cluster_item_matrix = df.groupby('Customer_Type')[
    [col for col in df.columns if 'department' in col]
].sum()

#### Normalización de la matriz

In [16]:
scaler = MinMaxScaler()
cluster_item_matrix_scaled = pd.DataFrame(
    scaler.fit_transform(cluster_item_matrix),
    index=cluster_item_matrix.index,
    columns=cluster_item_matrix.columns
)

Agrupamos los datos por Customer_Type (que refiere a los 4 diferentes clusters).
Sumamos las interacciones (número de productos comprados en cada departamento) para cada cluster.
Normalizamos los valores de la matriz para que estén en un rango de 0 a 1. Esto asegura que las columnas (departamentos) tengan un impacto equilibrado en los cálculos posteriores.

Este paso nos permite convertir el dataset en una matriz donde:

Las filas representan clusters (Customer_Type).
Las columnas representan departamentos.
Los valores son interacciones normalizadas (0 a 1).

#### Similitud entre departamentos

In [17]:
item_similarity = cosine_similarity(cluster_item_matrix_scaled.T)
item_similarity_df = pd.DataFrame(
    item_similarity, 
    index=cluster_item_matrix_scaled.columns, 
    columns=cluster_item_matrix_scaled.columns
)

Calculamos la similitud entre las columnas (departamentos) usando cosine similarity.
Convertimos la salida en un DataFrame llamado item_similarity_df para que sea más fácil de usar.

Cosine Similarity: La similitud coseno mide cuán similares son dos vectores en un espacio multidimensional. En este caso, queremos saber qué departamentos tienen patrones de interacción similares según los clusters.

Con este procedimiento obtenemos una matriz de similitud donde:

Las filas y columnas representan departamentos.
Los valores indican cuán similares son los departamentos (entre 0 y 1).

#### Recomendaciones basadas en el cluster

In [ ]:
def recommend_items_for_cluster(cluster, cluster_item_matrix, item_similarity_df, top_n=5):
    cluster_vector = cluster_item_matrix.loc[cluster]
    scores = cluster_vector @ item_similarity_df
    scores = scores.sort_values(ascending=False)
    interacted_items = cluster_vector > 0
    recommendations = scores[~interacted_items].head(top_n)

    return recommendations.index.tolist()

##### Explicación de la función generada:

Input:

cluster: El cluster para el que queremos generar recomendaciones.

cluster_item_matrix: Matriz de interacción de clusters.

item_similarity_df: Matriz de similitud entre departamentos.

top_n: Número de ítems (departamentos) a recomendar.

Cálculo de Puntuaciones:

Multiplicamos el vector del cluster por la matriz de similitud (cluster_vector @ item_similarity_df). Esto genera una puntuación para cada departamento basada en su similitud con los departamentos que el cluster ya ha interactuado.

Ordenar y Filtrar:

Ordenamos los departamentos por relevancia (puntuación). Eliminamos los departamentos con los que el cluster ya ha interactuado.

Recomendaciones:
Seleccionamos los top_n departamentos con las puntuaciones más altas para finalmente obtener una lista de los mejores departamentos recomendados para un cluster específico.

#### Ejemplo de recomendación para un cluster específico

In [ ]:
cluster_example = cluster_item_matrix.index[0]
recommendations = recommend_items_for_cluster(cluster_example, cluster_item_matrix_scaled, item_similarity_df)
print(f"Recomendaciones para el cluster {cluster_example}: {recommendations}")

Recomendaciones para el cluster Básicos Frecuentes: ['department_babies', 'department_bakery', 'department_produce', 'department_pets', 'department_personal care']


Seleccionamos un cluster (cluster_example) de la matriz de interacción.

Llamamos a la función recommend_items_for_cluster para obtener las recomendaciones para este cluster, esto genera una lista de los mejores departamentos recomendados para el cluster seleccionado.

### VALIDACION DE MODELO DE RECOMENDACION

#### Se dividen datos en entrenamiento y prueba (a nivel de cluster)

In [ ]:
def train_test_split_matrix(matrix, test_size=0.2, random_state=42):
    np.random.seed(random_state)

    train = matrix.copy()
    test = matrix.copy()
    for cluster in matrix.index:
        interactions = matrix.loc[cluster]
        non_zero_indices = interactions[interactions > 0].index
        test_indices = np.random.choice(
            non_zero_indices, 
            size=int(test_size * len(non_zero_indices)), 
            replace=False
        )
        train.loc[cluster, test_indices] = 0
        test.loc[cluster, ~test.columns.isin(test_indices)] = 0

    return train, test

In [30]:
train_matrix, test_matrix = train_test_split_matrix(cluster_item_matrix_scaled)

#### Resumen de las matrices

In [ ]:
print("Matriz de entrenamiento:")
print(train_matrix.to_string(max_rows=10, max_cols=30))

print("\nMatriz de prueba:")
print(test_matrix.to_string(max_rows=10, max_cols=30))

Matriz de entrenamiento:
                    department_babies  department_bakery  department_beverages  department_breakfast  department_bulk  department_canned goods  department_dairy eggs  department_deli  department_dry goods pasta  department_frozen  department_household  department_international  department_meat seafood  department_missing  department_other  department_pantry  department_personal care  department_pets  department_produce  department_snacks
Customer_Type                                                                                                                                                                                                                                                                                                                                                                                                                                            
Básicos Frecuentes           0.000000           0.000000              0.000000             

#### Filtrar datos de prueba

In [35]:
def filter_test_data(predicted, test_data):
    test_indices = test_data > 0
    return predicted[test_indices], test_data[test_indices]

#### Predicción y cálculo de RMSE

In [ ]:
predicted_matrix = train_matrix @ item_similarity_df
predicted_values, actual_values = filter_test_data(predicted_matrix, test_matrix)
rmse = np.sqrt(mean_squared_error(actual_values, predicted_values))
print(f"RMSE del modelo: {rmse:.4f}")